In [ ]:
import os
import json
from pprint import pprint

In [ ]:
with open("data/cleaned/sample/businesses.json") as f:
    businesses_original = [json.loads(line) for line in f]
with open("data/cleaned/sample/reviews.json") as f:
    reviews_original = [json.loads(line) for line in f]
with open("data/cleaned/sample/users.json") as f:
    users_original = [json.loads(line) for line in f]

In [ ]:
businesses = {b["business_id"]: b for b in businesses_original}
reviews = {r["review_id"]: r for r in reviews_original}
users = {u["user_id"]: u for u in users_original}

In [ ]:
user_reviews = [review for review in reviews if review["user_id"] == user_id]

In [ ]:
user_reviews